In [16]:
# 1. Importar librerías
import pandas as pd
import numpy as np
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display
import plotly.io as pio

# 2. Configurar Plotly para que abra en el navegador
pio.renderers.default = "browser"

# 3. Cargar los archivos CSV
df_alumnos = pd.read_csv('csv/alumnos.csv', encoding='utf-8')
df_carreras = pd.read_csv('csv/carreras.csv', encoding='utf-8')
df_predicciones = pd.read_csv('csv/predicciones.csv', encoding='utf-8')

# 4. Renombrar carreras para claridad
df_carreras.rename(columns={'nombre': 'nombre_carrera'}, inplace=True)

# 5. Convertir id_alumno a tipo numérico
df_alumnos['id_alumno'] = pd.to_numeric(df_alumnos['id_alumno'], errors='coerce').astype('Int64')
df_predicciones['id_alumno'] = pd.to_numeric(df_predicciones['id_alumno'], errors='coerce').astype('Int64')

# 6. Unir alumnos con carreras
df_merged = df_alumnos.merge(
    df_carreras[['_id', 'nombre_carrera']],
    left_on='id_carrera',
    right_on='_id',
    how='left'
)

# 7. Unir con predicciones
df_merged = df_merged.merge(
    df_predicciones[['id_alumno', 'riesgo']],
    on='id_alumno',
    how='left'
)

# 8. Clasificar el riesgo
condiciones = [
    df_merged['riesgo'] == 'Alto',
    df_merged['riesgo'] == 'Medio',
    df_merged['riesgo'] == 'Bajo'
]
valores = ['Alto', 'Medio', 'Bajo']
df_merged['Clasificacion_riesgo'] = np.select(condiciones, valores, default='No especificado')

# 9. Preparar datos para visualización
orden = ['Bajo', 'Medio', 'Alto']
conteo = df_merged['Clasificacion_riesgo'].value_counts().reindex(orden).fillna(0)
df_conteo = conteo.reset_index()
df_conteo.columns = ['Categoría', 'Cantidad']

# 10. Crear gráfico de barras
fig = px.bar(
    df_conteo,
    x='Categoría',
    y='Cantidad',
    text='Cantidad',
    color='Categoría',
    title='Distribución de niveles de riesgo de estudiantes',
    color_discrete_sequence=['#77dd77', '#ffb347', '#ff6961']
)

fig.update_traces(textposition='outside')
fig.update_layout(
    xaxis_title='Nivel de riesgo',
    yaxis_title='Cantidad de estudiantes',
    uniformtext_minsize=10,
    uniformtext_mode='hide'
)

# 11. Mostrar gráfico (usando renderizador de navegador)
fig.show()